In [1]:
import dataflowkit.datasets as D
import dataflowkit.graphs as G
import dataflowkit.recipes as R
from dataflowkit.utils.print_time import print_time, print_end
from multiprocessing import Process, Queue, cpu_count
import threading, numpy as np

In [2]:
class DummyRecipe1(R.BaseRecipe):
    # Must be executed in singleton, maybe heavy memory demanding
    def execute(self, ins, outs):
        # Takes ~ 0.093899s
        for i in np.arange(1e6):
            pass

In [3]:
class DummyRecipe2(R.BaseRecipe):
    # Hevay calcuation process
    def execute(self, ins, outs):
        # Takes ~ 0.823933s
        for i in np.arange(1e7):
            pass

In [4]:
def process_job(file_queue, mrf, master_index):
    while not file_queue.empty():
        file = file_queue.get()
        print_time('master_index: ' + str(master_index) + ' file: ' + str(file))
        
        # declare recipes
        dummy_recipe1 = mrf.create(DummyRecipe1, no_slaves=1)
        dummy_recipe2 = DummyRecipe2()
        
        # declare datasets
        dataset1 = D.InMemory()
        dataset2 = D.InMemory()
        dataset3 = D.InMemory()
        
        # declare graph
        dummy_recipe1.execute(ins=dict(dataset1=dataset1), outs=dict(dataset2=dataset2))
        dummy_recipe2.execute(ins=dict(dataset2=dataset2), outs=dict(dataset3=dataset3))
        
        print_end('master_index: ' + str(master_index) + ' file: ' + str(file))

#### Declare the files be process

In [5]:
file_queue = Queue()
files = np.arange(20)
for file in files:
    file_queue.put(file)

#### MasterRecipeFactory must be initialized before opeining the processes

In [6]:
no_master = 4
mrf = G.MasterRecipeFactory.get_inatance()

#### Declare the processes

In [7]:
processes = [Process(target=process_job, args=(file_queue, mrf, master_index)) 
             for master_index in np.arange(no_master)]

#### Execute the processes

In [8]:
for p in processes:
    p.start()

master_index: 0 file: 0 1.674101
master_index: 2 file: 2 1.722167
master_index: 3 file: 3 1.783775
master_index: 1 file: 1 1.954055
master_index: 0 file: 4 1.303322
master_index: 2 file: 5 1.330583
master_index: 3 file: 6 1.438483
master_index: 1 file: 7 1.519969
master_index: 0 file: 8 1.277872
master_index: 2 file: 9 1.301801
master_index: 3 file: 10 1.363171
master_index: 1 file: 11 1.563292
master_index: 0 file: 12 1.496758
master_index: 2 file: 13 1.444347
master_index: 3 file: 14 1.342112
master_index: 1 file: 15 1.342489
master_index: 0 file: 16 1.29934
master_index: 2 file: 17 1.292061
master_index: 3 file: 18 1.168567
master_index: 1 file: 19 0.957718
